In [2]:
import seaborn as sns
sns.set(rc={'axes.facecolor': 'dimgrey', 'grid.color': 'lightgrey'})
import pandas as pd
import torch
import copy
from torch_geometric.loader import NeighborLoader
import pickle

In [4]:
%run utils.ipynb
%run gcn_model.ipynb
%run GAT_model.ipynb
%run FeatureAggregationModel.ipynb
%run LSTM_GNN.ipynb

In [5]:

# class 1 is illicit, 2 is licit, unknown is unknown
labels = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
edges = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
nodes = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)

In [6]:
indexes = labels.index[labels["class"] != "unknown"].tolist()
new_labels = labels.replace("unknown", 3)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
data, graph_info = time_step_split(nodes, edges, new_labels, device)
for key in data:
    print(key, len(data[key]))

train 31
val 5
test 13


In [14]:

from torch_geometric.data import DataLoader
from torch_geometric.loader import ClusterLoader, RandomNodeLoader, ImbalancedSampler
print(data["train"])
sampler = ImbalancedSampler(data["train"])
train_loader = DataLoader(data["train"], sampler=sampler)

[Data(x=[7880, 165], edge_index=[2, 9164], y=[7880]), Data(x=[4544, 165], edge_index=[2, 5241], y=[4544]), Data(x=[6621, 165], edge_index=[2, 8316], y=[6621]), Data(x=[5693, 165], edge_index=[2, 8180], y=[5693]), Data(x=[6803, 165], edge_index=[2, 8623], y=[6803]), Data(x=[4328, 165], edge_index=[2, 5242], y=[4328]), Data(x=[6048, 165], edge_index=[2, 7253], y=[6048]), Data(x=[4457, 165], edge_index=[2, 5186], y=[4457]), Data(x=[4996, 165], edge_index=[2, 5939], y=[4996]), Data(x=[6727, 165], edge_index=[2, 8588], y=[6727]), Data(x=[4296, 165], edge_index=[2, 4656], y=[4296]), Data(x=[2047, 165], edge_index=[2, 2213], y=[2047]), Data(x=[4528, 165], edge_index=[2, 4827], y=[4528]), Data(x=[2022, 165], edge_index=[2, 2078], y=[2022]), Data(x=[3639, 165], edge_index=[2, 3823], y=[3639]), Data(x=[2975, 165], edge_index=[2, 3120], y=[2975]), Data(x=[3385, 165], edge_index=[2, 3650], y=[3385]), Data(x=[1976, 165], edge_index=[2, 2115], y=[1976]), Data(x=[3506, 165], edge_index=[2, 3838], y=[

AssertionError: 

In [17]:
mega_results_gcn = []
for n in range(10):
    print(n)
    GCN = GNN_Example(node_dim=data["train"][0].x.shape[1], edge_dim=1, output_dim=2, hidden_dim=256, n_gnn_layers=3, K=2, dropout_rate=0.1).to(device)
    lr = 1e-3
    epochs = 100
    # weights for classes
    weight = torch.tensor([0.5, 0.5]).to(device)
    optimizer = torch.optim.Adam(GCN.parameters(), lr=lr)
    loss_fn = torch.nn.NLLLoss(weight)

    best_model = None
    best_test_loss = 1e10
    best_result = None
    losses = []
    test_losses = []
    best_valid_auc= 0
    for epoch in range(1, 1 + epochs):
        # train with random split
        loss = train(GCN, data['train'], optimizer, loss_fn, device)
        losses.append(loss)
        result = test(GCN, data, device)
        train_acc, val_acc, test_acc, train_auc, val_auc, test_auc = result
        if val_auc > best_valid_auc:
            best_valid_auc = val_auc
            best_model = copy.deepcopy(GCN)
            best_result = [train_acc, val_acc, test_acc, train_auc, val_auc, test_auc]
        # if epoch % 10 == 0:
        #     print('Epoch: {:02},'.format(epoch),
        #           'Loss:{:.4f}'.format(loss),
        #           'Train:\n{}\n'.format(train_acc),
        #           'Train_auc_roc: {}'.format(train_auc),
        #           '\n\n'
        #           'Valid:\n{}\n'.format(val_acc),
        #           'Val_auc_roc: {}'.format(val_auc),
        #           '\n\n'
        #           'Test:\n{}\n'.format(test_acc),
        #           'Test_auc_roc: {}'.format(test_auc),
        #           '\n'
        #           )
    state = {"net":best_model.state_dict(), "optimizer":optimizer.state_dict()}
    torch.save(state, "gcn.pt")
    mega_results_gcn.append(best_result)
    print(best_result)


0
[                   0           1  accuracy   macro avg  weighted avg
precision   0.766105    0.978105  0.969443    0.872105      0.968460
recall      0.715185    0.984999  0.969443    0.850092      0.969443
f1-score    0.732893    0.981477  0.969443    0.857185      0.968573
support    98.709677  792.096774  0.969443  890.806452    890.806452,                     0           1  accuracy    macro avg  weighted avg
precision    0.532961    0.983098  0.927905     0.758029      0.950288
recall       0.858724    0.933248  0.927905     0.895986      0.927905
f1-score     0.631892    0.957464  0.927905     0.794678      0.936148
support    123.400000  942.200000  0.927905  1065.600000   1065.600000,                    0           1  accuracy    macro avg  weighted avg
precision   0.247448    0.966387  0.896466     0.606917      0.927532
recall      0.367032    0.921225  0.896466     0.644129      0.896466
f1-score    0.284285    0.942567  0.896466     0.613426      0.908924
support    66.7

In [18]:
with open("mega_results_gcn", "wb") as fp:   #Pickling
    pickle.dump(mega_results_gcn, fp)

In [20]:
%run GAT_model.ipynb
mega_results_gat = []
for n in range(10):
    print(n)
    GAT = GraphAttentionNetwork(node_dim=data["train"][0].x.shape[1], output_dim=2, hidden_dim=256, n_gnn_layers=1, heads=2, dropout_rate=0.5).to(device)
    lr = 1e-3
    epochs = 100
    # weights for classes
    weight = torch.tensor([0.5, 0.5]).to(device)
    optimizer = torch.optim.Adam(GAT.parameters(), lr=lr)
    loss_fn = torch.nn.NLLLoss(weight)

    best_model = None
    best_valid_auc = 0
    best_result = None
    losses = []

    for epoch in range(1, 1 + epochs):
        # train with random split
        loss = train(GAT, data['train'], optimizer, loss_fn, device)
        losses.append(loss)
        result = test(GAT, data, device)
        train_acc, val_acc, test_acc, train_auc, val_auc, test_auc = result
        if val_auc > best_valid_auc:
            best_valid_auc = val_auc
            best_model = copy.deepcopy(GAT)
            best_result = [train_acc, val_acc, test_acc, train_auc, val_auc, test_auc]
    state = {"net":best_model.state_dict(), "optimizer":optimizer.state_dict()}
    torch.save(state, "gat.pt")
    mega_results_gat.append(best_result)
    print(best_result)

0
[                   0           1  accuracy   macro avg  weighted avg
precision   0.647941    0.952959  0.930031    0.800450      0.931263
recall      0.551229    0.961270  0.930031    0.756249      0.930031
f1-score    0.567994    0.956574  0.930031    0.762284      0.928436
support    98.709677  792.096774  0.930031  890.806452    890.806452,                     0           1  accuracy    macro avg  weighted avg
precision    0.360194    0.982729  0.852587     0.671461      0.928504
recall       0.881021    0.843063  0.852587     0.862042      0.852587
f1-score     0.492518    0.906967  0.852587     0.699742      0.877357
support    123.400000  942.200000  0.852587  1065.600000   1065.600000,                    0           1  accuracy    macro avg  weighted avg
precision   0.193617    0.970532  0.796739     0.582075      0.926731
recall      0.504761    0.809792  0.796739     0.657276      0.796739
f1-score    0.261865    0.879176  0.796739     0.570520      0.846340
support    66.7

In [21]:
with open("mega_results_gat", "wb") as fp:   #Pickling
    pickle.dump(mega_results_gat, fp)

In [22]:
mega_results_sfa = []
for n in range(10):
    print(n)
    SFA = SpatialFeatureAggregator(node_dim=data["train"][0].x.shape[1], edge_dim=1, output_dim=2, hidden_dim=256, n_gnn_layers=3, K=2, dropout_rate=0.1).to(device)
    lr = 1e-3
    epochs = 100
    # weights for classes
    weight = torch.tensor([0.5, 0.5]).to(device)
    optimizer = torch.optim.Adam(SFA.parameters(), lr=lr)
    loss_fn = torch.nn.NLLLoss(weight)

    best_model = None
    best_valid_auc = 0
    best_result = None
    losses = []
    best_test_loss = 0


    for epoch in range(1, 1 + epochs):
        # train with random split
        loss = train(SFA, data['train'], optimizer, loss_fn, device)
        losses.append(loss)
        result = test(SFA, data, device)
        train_acc, val_acc, test_acc, train_auc, val_auc, test_auc = result
        if val_auc > best_valid_auc:
            best_valid_auc = val_auc
            best_model = copy.deepcopy(SFA)
            best_result = [train_acc, val_acc, test_acc, train_auc, val_auc, test_auc]
    state = {"net":best_model.state_dict(), "optimizer":optimizer.state_dict()}
    torch.save(state, "SFA.pt")
    mega_results_sfa.append(best_result)

0
1
2
3
4
5
6
7
8
9


In [23]:
with open("mega_results_sfa", "wb") as fp:   #Pickling
    pickle.dump(mega_results_sfa, fp)

In [24]:
mega_results_tb =[]
for n in range(10):
    TB = TemporalBoy(node_dim=data["train"][0].x.shape[1], edge_dim=1, output_dim=2, hidden_dim=256, n_gnn_layers=3, K=2, dropout_rate=0.1).to(device)
    lr = 1e-3
    epochs = 100
    # weights for classes
    weight = torch.tensor([0.5, 0.5]).to(device)
    optimizer = torch.optim.Adam(TB.parameters(), lr=lr)
    loss_fn = torch.nn.NLLLoss(weight)

    best_model = None
    best_valid_auc = 0
    best_result = None
    losses = []
    best_test_loss = 0

    for epoch in range(1, 1 + epochs):
        # train with random split
        loss = train(TB, data['train'], optimizer, loss_fn, device)
        losses.append(loss)
        result = test(TB, data, device)
        train_acc, val_acc, test_acc, train_auc, val_auc, test_auc = result
        if val_auc > best_valid_auc:
            best_valid_auc = val_auc
            best_model = copy.deepcopy(TB)
            best_result = [train_acc, val_acc, test_acc, train_auc, val_auc, test_auc]
        # if epoch % 10 == 0:
        #     print("Epoch: ", epoch)
        #     for item, amount in result.items():
        #             print("{} ({})".format(item, amount))
        #     print("############################")
    state = {"net":best_model.state_dict(), "optimizer":optimizer.state_dict()}
    torch.save(state, "TB.pt")
    mega_results_tb.append(best_result)

D:\Python\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
D:\Python\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
D:\Python\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
D:\Python\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent l

In [25]:
with open("mega_results_tb", "wb") as fp:   #Pickling
    pickle.dump(mega_results_tb, fp)

In [72]:
import numpy as np

def mean_std_results(results):
    f1_0 = []
    f1_1 = []
    f1_avg = []
    for result in results:
        f1_0.append(result[2]["0"]["f1-score"])
        f1_1.append(result[2]["1"]["f1-score"])
        f1_avg.append(result[2]["macro avg"]["f1-score"])
    return (np.mean(f1_0), np.std(f1_0)),(np.mean(f1_1),np.std(f1_1)),(np.mean(f1_avg),np.std(f1_avg)),

In [75]:
mean_std_results(mega_results_gcn)

((0.29082152955234253, 0.007263991102777121),
 (0.9401719702666735, 0.004940283751679395),
 (0.6154967499095081, 0.005738649192763499))

In [74]:
mean_std_results(mega_results_gat)

((0.25892677337408615, 0.014150142688408661),
 (0.8881142419917909, 0.013180160608831802),
 (0.5735205076829384, 0.013015052939925761))

In [76]:
mean_std_results(mega_results_sfa)


((0.30332387876084965, 0.006462005093711699),
 (0.9448299412616554, 0.004954678576242759),
 (0.6240769100112525, 0.005096571747988077))

In [77]:
mean_std_results(mega_results_tb)


((0.27380298791921376, 0.010335485580295128),
 (0.944866063637037, 0.005261707735817781),
 (0.6093345257781255, 0.007520315943807319))